# PyTorch: Classify Binary

![houses](../images/mines.png)

In [3]:
import torch
import torch.nn as nn
from torch import optim
import torchmetrics

from sklearn.preprocessing import LabelBinarizer, PowerTransformer

import aiqc
from aiqc import datum

---

## Example Data

Reference [Example Datasets](example_datasets.ipynb) for more information.

In [4]:
df = datum.to_pandas('sonar.csv')

In [5]:
df.head()

,a,b,c,d,e,f,g,h,i,j,...,az,ba,bb,bc,bd,be,bf,bg,bh,object
0,0.0200,0.0371,0.0428,0.0207,0.0954,0.0986,0.1539,0.1601,0.3109,0.2111,...,0.0027,0.0065,0.0159,0.0072,0.0167,0.0180,0.0084,0.0090,0.0032,R
1,0.0453,0.0523,0.0843,0.0689,0.1183,0.2583,0.2156,0.3481,0.3337,0.2872,...,0.0084,0.0089,0.0048,0.0094,0.0191,0.0140,0.0049,0.0052,0.0044,R
2,0.0262,0.0582,0.1099,0.1083,0.0974,0.2280,0.2431,0.3771,0.5598,0.6194,...,0.0232,0.0166,0.0095,0.0180,0.0244,0.0316,0.0164,0.0095,0.0078,R
3,0.0100,0.0171,0.0623,0.0205,0.0205,0.0368,0.1098,0.1276,0.0598,0.1264,...,0.0121,0.0036,0.0150,0.0085,0.0073,0.0050,0.0044,0.0040,0.0117,R
4,0.0762,0.0666,0.0481,0.0394,0.0590,0.0649,0.1209,0.2467,0.3564,0.4459,...,0.0031,0.0054,0.0105,0.0110,0.0015,0.0072,0.0048,0.0107,0.0094,R


---

## a) High-Level API

Reference [High-Level API Docs](api_high_level.ipynb) for more information including how to work with non-tabular data.

In [6]:
splitset = aiqc.Pipeline.Tabular.make(
    dataFrame_or_filePath = df
    , label_column = 'object'
    , size_test = 0.22
    , size_validation = 0.12
    , label_encoder = LabelBinarizer(sparse_output=False)
    , feature_encoders = [{
        "sklearn_preprocess": PowerTransformer(method='yeo-johnson', copy=False)
        , "dtypes": ['float64']
    }]
    
    , dtype = None
    , features_excluded = None
    , fold_count = None
    , bin_count = None
)


___/ featurecoder_index: 0 \_________

=> The column(s) below matched your filter(s) and were ran through a test-encoding successfully.
['a',
 'b',
 'c',
 'd',
 'e',
 'f',
 'g',
 'h',
 'i',
 'j',
 'k',
 'l',
 'm',
 'n',
 'o',
 'p',
 'q',
 'r',
 's',
 't',
 'u',
 'v',
 'w',
 'x',
 'y',
 'z',
 'aa',
 'ab',
 'ac',
 'ad',
 'ae',
 'af',
 'ag',
 'ah',
 'ai',
 'aj',
 'ak',
 'al',
 'am',
 'an',
 'ao',
 'ap',
 'aq',
 'ar',
 'as',
 'at',
 'au',
 'av',
 'aw',
 'ax',
 'ay',
 'az',
 'ba',
 'bb',
 'bc',
 'bd',
 'be',
 'bf',
 'bg',
 'bh']

=> Nice! Now all feature column(s) have encoder(s) associated with them.
No more Featurecoders can be added to this Encoderset.



In [7]:
def fn_build(features_shape, label_shape, **hp):
    model = nn.Sequential(
        nn.Linear(features_shape[0], 12),
        nn.BatchNorm1d(12,12),
        nn.ReLU(),
        nn.Dropout(p=0.5),

        nn.Linear(12, label_shape[0]),
        nn.Sigmoid()
    )
    return model

In [8]:
def fn_train(model, loser, optimizer, samples_train, samples_evaluate, **hp):
    ## --- Prepare mini batches for analysis ---
    batched_features, batched_labels = aiqc.torch_batch_splitter(
        samples_train['features'], samples_train['labels'],
        batch_size=5, enforce_sameSize=False, allow_1Sample=False
    )

    ## --- Metrics ---
    acc = torchmetrics.Accuracy()
    # Mirrors `keras.model.History.history` object.
    history = {
        'loss':list(), 'accuracy': list(), 
        'val_loss':list(), 'val_accuracy':list()
    }

    ## --- Training loop ---
    epochs = hp['epoch_count']
    for epoch in range(epochs):
        ## --- Batch training ---
        for i, batch in enumerate(batched_features):      
            # Make raw (unlabeled) predictions.
            batch_probability = model(batched_features[i])
            batch_loss = loser(batch_probability, batched_labels[i])
            # Backpropagation.
            optimizer.zero_grad()
            batch_loss.backward()
            optimizer.step()

        ## --- Epoch metrics ---
        # Overall performance on training data.
        train_probability = model(samples_train['features'])
        train_loss = loser(train_probability, samples_train['labels'])
        train_acc = acc(train_probability, samples_train['labels'].to(torch.short))
        history['loss'].append(float(train_loss))
        history['accuracy'].append(float(train_acc))
        # Performance on evaluation data.
        eval_probability = model(samples_evaluate['features'])
        eval_loss = loser(eval_probability, samples_evaluate['labels'])
        eval_acc = acc(eval_probability, samples_evaluate['labels'].to(torch.short))    
        history['val_loss'].append(float(eval_loss))
        history['val_accuracy'].append(float(eval_acc))
    return model, history

Optional, will be automatically selected based on `analysis_type` if left as `None`.

In [9]:
def fn_optimize(model, **hp):
    optimizer = optim.Adamax(
        model.parameters()
        , lr=hp['learning_rate']
    )
    return optimizer

In [10]:
hyperparameters = {
    "learning_rate": [0.01, 0.005]
    , "epoch_count": [50]
}

In [11]:
queue = aiqc.Experiment.make(
    library = "pytorch"
    , analysis_type = "classification_binary"
    , fn_build = fn_build
    , fn_train = fn_train
    , splitset_id = splitset.id
    , encoderset_id = splitset.encodersets[0]
    , repeat_count = 2
    , hide_test = False
    , hyperparameters = hyperparameters
    
    , fn_lose = None #optional/ automated
    , fn_optimize = fn_optimize #optional/ automated
    , fn_predict = None #optional/ automated
    , foldset_id = None
)

In [12]:
queue.run_jobs()

🔮 Training Models 🔮: 100%|██████████████████████████████████████████| 4/4 [00:09<00:00,  2.35s/it]


For more information on visualization of performance metrics, reference the [Visualization & Metrics](visualization.html) documentation.

---

## b) Low-Level API

Reference [Low-Level API Docs](api_high_level.ipynb) for more information including how to work with non-tabular data and defining optimizers.

In [13]:
dataset = aiqc.Dataset.Tabular.from_pandas(df)

In [14]:
label_column = 'object'

In [15]:
label = dataset.make_label(columns=[label_column])

In [16]:
featureset = dataset.make_featureset(exclude_columns=[label_column])

In [17]:
splitset = featureset.make_splitset(
    label_id = label.id
    , size_test = 0.22
    , size_validation = 0.12
)

In [18]:
encoderset = splitset.make_encoderset()

In [19]:
labelcoder = encoderset.make_labelcoder(
    sklearn_preprocess = LabelBinarizer(sparse_output=False)
)

In [20]:
featurecoder_0 = encoderset.make_featurecoder(
    sklearn_preprocess = PowerTransformer(method='yeo-johnson', copy=False)
    , dtypes = ['float64']
)


___/ featurecoder_index: 0 \_________

=> The column(s) below matched your filter(s) and were ran through a test-encoding successfully.
['a',
 'b',
 'c',
 'd',
 'e',
 'f',
 'g',
 'h',
 'i',
 'j',
 'k',
 'l',
 'm',
 'n',
 'o',
 'p',
 'q',
 'r',
 's',
 't',
 'u',
 'v',
 'w',
 'x',
 'y',
 'z',
 'aa',
 'ab',
 'ac',
 'ad',
 'ae',
 'af',
 'ag',
 'ah',
 'ai',
 'aj',
 'ak',
 'al',
 'am',
 'an',
 'ao',
 'ap',
 'aq',
 'ar',
 'as',
 'at',
 'au',
 'av',
 'aw',
 'ax',
 'ay',
 'az',
 'ba',
 'bb',
 'bc',
 'bd',
 'be',
 'bf',
 'bg',
 'bh']

=> Nice! Now all feature column(s) have encoder(s) associated with them.
No more Featurecoders can be added to this Encoderset.



In [21]:
def fn_build(features_shape, label_shape, **hp):
    model = nn.Sequential(
        nn.Linear(features_shape[0], 12),
        nn.BatchNorm1d(12,12),
        nn.ReLU(),
        nn.Dropout(p=0.5),

        nn.Linear(12, label_shape[0]),
        nn.Sigmoid()
    )
    return model

In [22]:
def fn_train(model, loser, optimizer, samples_train, samples_evaluate, **hp):
    ## --- Prepare mini batches for analysis ---
    batched_features, batched_labels = aiqc.torch_batch_splitter(
        samples_train['features'], samples_train['labels'],
        batch_size=5, enforce_sameSize=False, allow_1Sample=False
    )

    ## --- Metrics ---
    acc = torchmetrics.Accuracy()
    # Mirrors `keras.model.History.history` object.
    history = {
        'loss':list(), 'accuracy': list(), 
        'val_loss':list(), 'val_accuracy':list()
    }

    ## --- Training loop ---
    epochs = hp['epoch_count']
    for epoch in range(epochs):
        ## --- Batch training ---
        for i, batch in enumerate(batched_features):      
            # Make raw (unlabeled) predictions.
            batch_probability = model(batched_features[i])
            batch_loss = loser(batch_probability, batched_labels[i])
            # Backpropagation.
            optimizer.zero_grad()
            batch_loss.backward()
            optimizer.step()

        ## --- Epoch metrics ---
        # Overall performance on training data.
        train_probability = model(samples_train['features'])
        train_loss = loser(train_probability, samples_train['labels'])
        train_acc = acc(train_probability, samples_train['labels'].to(torch.short))
        history['loss'].append(float(train_loss))
        history['accuracy'].append(float(train_acc))
        # Performance on evaluation data.
        eval_probability = model(samples_evaluate['features'])
        eval_loss = loser(eval_probability, samples_evaluate['labels'])
        eval_acc = acc(eval_probability, samples_evaluate['labels'].to(torch.short))    
        history['val_loss'].append(float(eval_loss))
        history['val_accuracy'].append(float(eval_acc))
    return model, history

Optional, will be automatically selected based on `analysis_type` if left as `None`.

In [23]:
def fn_optimize(model, **hp):
    optimizer = optim.Adamax(
        model.parameters()
        , lr=hp['learning_rate']
    )
    return optimizer

In [24]:
hyperparameters = {
    "learning_rate": [0.01, 0.005]
    , "epoch_count": [50]
}

In [25]:
algorithm = aiqc.Algorithm.make(
    library = "pytorch"
    , analysis_type = "classification_binary"
    , fn_build = fn_build
    , fn_train = fn_train
    , fn_optimize = fn_optimize
)

In [26]:
hyperparamset = algorithm.make_hyperparamset(
    hyperparameters = hyperparameters
)

In [27]:
queue = algorithm.make_queue(
    splitset_id = splitset.id
    , hyperparamset_id = hyperparamset.id
    , encoderset_id  = encoderset.id
    , repeat_count = 1
)

In [28]:
queue.run_jobs()

🔮 Training Models 🔮: 100%|██████████████████████████████████████████| 2/2 [00:04<00:00,  2.34s/it]


For more information on visualization of performance metrics, reference the [Visualization & Metrics](visualization.html) documentation.